In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from PIL import Image
from keras.models import load_model
from keras.metrics import AUC
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D,GlobalAveragePooling2D,BatchNormalization,Activation
from tensorflow.keras import backend as K

In [ ]:
train_dir= '../input/plant-pathology-2021-fgvc8/train_images'
test_dir =  '../input/plant-pathology-2021-fgvc8/test_images'
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
train.head()

In [ ]:
train = pd.DataFrame(train,columns = ['image','labels'])

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(20,12))
labels = sns.barplot(train.labels.value_counts().index,train.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

In [ ]:
#Separate the label base on space charactor
train['labels'] = train['labels'].apply(lambda s: s.split(' '))
train[:10]

In [ ]:
def add_gauss_noise(x,sigma2=0.05):
    return x+np.random.normal(0, sigma2, x.shape)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 10,#Performing Rotation
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = None,
    shear_range = 0.1,
    zoom_range = 0.1,
    rescale = 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    
    validation_split= 0.1
)
bsize = 16

In [ ]:
train_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
valid_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
from tensorflow.keras.preprocessing import image
import random

K.clear_session()
random.seed(4487); tf.random.set_seed(4487)

input_shape= (224,224,3)
# 
base_model = ResNet50V2(input_shape=input_shape, include_top=False,weights= "imagenet")

In [ ]:
from tensorflow.keras.layers import MaxPooling2D,GlobalAveragePooling2D,BatchNormalization,Activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(inputs=base_model.input ,outputs = predictions)

In [ ]:
from tensorflow.keras.models import Sequential, Model

model = tf.keras.Model(inputs=base_model.input ,outputs = predictions)

f1 = tfa.metrics.F1Score(num_classes=6, average='macro')
model.compile(optimizer=keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True), 
              loss=tfa.losses.SigmoidFocalCrossEntropy(), metrics=[f1])

lrschedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.05, patience=5, verbose=1)
callbacks_list = [lrschedule]

history = model.fit(
            train_data,
            validation_data=valid_data, 
            epochs=15,
            callbacks=callbacks_list, 
            verbose=1)

In [ ]:
loss, f1score = model.evaluate_generator(valid_data,verbose=1)

In [ ]:
model.save('ResNet 50 v2.h5')